<a href="https://colab.research.google.com/github/riteshpen/Fake-Real_News/blob/main/code/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: connect to drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install spacy
!python -m spacy download en_core_web_lg
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
import streamlit as st
import pandas as pd
import spacy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
import joblib

# Load the spaCy model
nlp = spacy.load("en_core_web_lg")

# Function to preprocess text
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)


In [ ]:
# Function to make a prediction
def predict_fake_news(headline, model, scaler):
    # Preprocess the input headline
    preprocessed_text = preprocess_text(headline)
    # Convert to vector
    vector = nlp(preprocessed_text).vector.reshape(1, -1)
    # Scale the vector
    vector_scaled = scaler.transform(vector)
    # Predict using the trained model
    prediction = model.predict(vector_scaled)
    return 'Real News' if prediction[0] == 1 else 'Fake News'


In [ ]:
df = pd.read_csv('/content/drive/My Drive/fake_news/WELFake_Dataset.csv')
df['preprocessed_text'] = df['title'].astype('str').apply(lambda x: preprocess_text(x))
df['vector'] = df['preprocessed_text'].apply(lambda x: nlp(x).vector)

# Prepare training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
          df.vector.values, df.label, test_size=0.2, random_state=2022, stratify=df.label
      )
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

# Scale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train model
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

joblib.dump(model, '/content/drive/My Drive/fake_news/gradient_boosting_model.pkl')
joblib.dump(scaler, '/content/drive/My Drive/fake_news/scaler.pkl')



['/content/drive/My Drive/fake_news/scaler.pkl']